In [17]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
from glob import glob
import cv2, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset

# %matplotlib inline
device = "cuda" if torch.cuda.is_available() else "cpu"

# !kaggle datasets download -d tongpython/cat-and-dog    # 下载数据集
# !mkdir content
# !unzip cat-and-dog.zip -d content


class cats_dogs_dataset(Dataset):
    def __init__(self, folder):
        cats = glob(folder + "/cats/*.jpg")
        dogs = glob(folder + "/dogs/*.jpg")
        self.fpaths = cats + dogs
        from random import shuffle, seed

        seed(10)
        shuffle(self.fpaths)
        self.targets = [
            fpath.split("/")[-1].startswith("dog") for fpath in self.fpaths
        ]  # dog=1 & cat=0

    def __len__(self):
        return len(self.fpaths)

    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = cv2.imread(f)[:, :, ::-1]
        im = cv2.resize(im, (224, 224))
        return torch.tensor(im / 255).permute(2, 0, 1).to(device).float(), torch.tensor(
            [target]
        ).float().to(device)


def conv_layer(ni, no, kernel_size, stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, no, kernel_size, stride),
        nn.ReLU(),
        nn.BatchNorm2d(no),
        nn.MaxPool2d(2),
    )


def get_model():
    model = nn.Sequential(
        conv_layer(3, 64, 3),
        conv_layer(64, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        nn.Flatten(),
        nn.Linear(512, 1),
        nn.Sigmoid(),
    ).to(device)
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer


def train_batch(x, y, model, optimizer, loss_fn):
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()


@torch.no_grad()
def accuracy(x, y, model):
    prediction = model(x)
    is_correct = (prediction > 0.5) == y
    return is_correct.cpu().numpy().tolist()


def get_data(train_data_dir, test_data_dir):

    train = cats_dogs_dataset(train_data_dir)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last=True)
    val = cats_dogs_dataset(test_data_dir)
    val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
    return trn_dl, val_dl


@torch.no_grad()
def val_loss(x, y, model, loss_fn):
    prediction = model(x)
    # global loss_fn
    val_loss = loss_fn(prediction, y)
    return val_loss.item()


def main():
    train_data_dir = "./content/training_set/training_set"
    test_data_dir = "./content/test_set/test_set"

    trn_dl, val_dl = get_data(train_data_dir, test_data_dir)
    model, loss_fn, optimizer = get_model()

    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in range(5):
        print(f"epoch: {epoch}")

        train_epoch_losses, train_epoch_accuracies = [], []
        val_epoch_accuracies = []
        val_epoch_losses = []

        # Training
        model.train()
        for ix, batch in enumerate(trn_dl):
            print(f"\ttraining: {ix}")
            x, y = batch

            batch_loss = train_batch(x, y, model, optimizer, loss_fn)
            train_epoch_losses.append(batch_loss)

            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)

        # Validation
        model.eval()
        for ix, batch in enumerate(val_dl):
            print(f"\tevaluating: {ix}")
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            val_epoch_accuracies.extend(val_is_correct)
            validation_loss = val_loss(x, y, model, loss_fn)
            val_epoch_losses.append(validation_loss)

        train_epoch_loss = np.mean(train_epoch_losses)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)
        val_epoch_loss = np.mean(val_epoch_losses)
        val_epoch_accuracy = np.mean(val_epoch_accuracies)

        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        val_losses.append(val_epoch_loss)
        val_accuracies.append(val_epoch_accuracy)

        print(
            f"Train Loss: {train_epoch_loss:.4f}, Train Accuracy: {train_epoch_accuracy:.4f}"
        )
        print(f"Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}")

In [18]:
main()

epoch: 0
	training: 0
	training: 1
	training: 2
	training: 3
	training: 4
	training: 5
	training: 6
	training: 7
	training: 8
	training: 9
	training: 10
	training: 11
	training: 12
	training: 13
	training: 14
	training: 15
	training: 16
	training: 17
	training: 18
	training: 19
	training: 20
	training: 21
	training: 22
	training: 23
	training: 24
	training: 25
	training: 26
	training: 27
	training: 28
	training: 29
	training: 30
	training: 31
	training: 32
	training: 33
	training: 34
	training: 35
	training: 36
	training: 37
	training: 38
	training: 39
	training: 40
	training: 41
	training: 42
	training: 43
	training: 44
	training: 45
	training: 46
	training: 47
	training: 48
	training: 49
	training: 50
	training: 51
	training: 52
	training: 53
	training: 54
	training: 55
	training: 56
	training: 57
	training: 58
	training: 59
	training: 60
	training: 61
	training: 62
	training: 63
	training: 64
	training: 65
	training: 66
	training: 67
	training: 68
	training: 69
	training: 70
	traini